In [1]:
import random

In [2]:
class Player:
    """
    Contains three methods
        1. setup_ships: takes no parameters and returns a list of lists showing the start position
                        of showing the start positions of your ships
                        Block of row i and column j, ship label 1-5 & 0 means no ship

        2. take_turn:   returns
                        -- a list of (row, column) tuples describing your shots
                        -- eg: [(1, 1), (1, 3), (1, 5), (2, 1) (4, 5)]
                        -- Note: should be equal to the number of ships present (check function)
                        -- or a tuple that has a ship number as the first element and the direction
                        -- of its moving value
                        -- 0: up, 1: right, 2: down, 3: left  (dict)

                        TODO: Do not take more shots than the number of ships
                        TODO: Do not move the ships outside the grid or radioactive zone

        3. get_name:   returns a string that is the name of the Player

    """
    import random
    from collections import defaultdict

    def __init__(self):

        self.player_battleground = [[0 for i in range(10)] for j in range(10)]
        self.opponent_battleground = [[0 for i in range(10)] for j in range(10)]

        self.ships_dd = Player.defaultdict(int)
        self.status = []

        self.enemy_dict = {(i, j): 0 for i in range(10) for j in range(10)}

        # number of turns
        self.counter = 0

    def setup_ships(self):
        '''

        :return: a 10x10 grid with the location of our ships
        '''
        grid = [[0 for i in range(10)] for j in range(10)]

        # ship numbers
        ship = [i for i in range(1, 6)]

        # randomly shuffle the ship list
        Player.random.shuffle(ship)

        for ship_number in ship:
            control = True

            while control:
                # randomly select a point on the grid
                ship_start_position = self.random_position()

                if self.vacant(ship_start_position, ship_number, grid):
                    for row in range(0, ship_number):
                        # print(f'{ship_start_position} ship : {ship_number}')
                        grid[ship_start_position[0] + row][ship_start_position[1]] = ship_number

                    control = False

        self.player_battleground = grid
        return grid

    def take_turn(self, history):
        '''

        :param history: records the game
                        list that contains one entry per turn
                        dict with 3 keys
                            - 'shots' : list of shots made in the turn
                            - 'hits' : an integer number of hits that your shots made
                            - 'incoming' : opponents list of shots, [] if moved

        :return: a list of tuple/s describing the shots or changed location
                depending on the strategy
        '''
        # last_shot = history[-1]['incoming']
        # TODO: use history to short circuit the take_turn function
        # TODO: consider if the opponent moves

        last_shot = history[-1]['incoming'] if self.counter > 0 else []

        self.player_battleground = self.update_opponent_shots(self.player_battleground, last_shot)

        # counting the number of ships left
        self.ships_dd = Player.defaultdict(int)  # reset count every time
        # ships_dd = Player.defaultdict(int)

        # returns the dict with the number of ships and hits
        for row in self.player_battleground:
            for point in row:
                self.ships_dd[point] += point

        self.status = self.ship_status(self.ships_dd)

        # Count the number of ships remaining
        number_of_ships = len(self.status) if self.status else []

        player_shots = history[-1]['shots'] if self.counter > 0 else []

        # updating the shots taken by the player
        # self.enemy_dict = self.update_enemy_dict(self.enemy_dict, player_shots)
        self.update_enemy_dict(player_shots)

        if self.counter > 0 and history[-1]['hits']:
            self.enemy_dict = self.minesweeper(self.enemy_dict, player_shots)
            shots_list = [number for number in self.shot_generator(self.enemy_dict, number_of_ships)]

        # elif self.counter > 0 or history[-1]['hits'] == 0:
        #     shots_list = [number for number in self.random_selection(self.enemy_dict, number_of_ships)]
            
        else:
            shots_list = [number for number in self.random_selection(self.enemy_dict, number_of_ships)]
            

        # update the number of turns
        self.counter += 1

        return shots_list

    def get_name(self):
        '''

        :return: string - name of the Player
        '''
        return "Syndicate_10"

    # function to select a point on the grid
    def random_position(self):
        return [Player.random.randint(0, 9), Player.random.randint(0, 9)]

    # function to check if the position is vacant
    def vacant(self, pos, ship, grid):
        # checking if the ship will fit in the board
        if pos[0] + ship <= 10:
            for i in range(0, ship):
                # checking if any other ship is already present
                if grid[pos[0] + i][pos[1]] != 0:
                    return False
            return True
        return False

    # function to shoot randomly with a given number of ships
    def random_shots(self, num=5):
        for x in range(0, num):
            yield Player.random.randint(0, 9), Player.random.randint(0, 9)

    # function to update the player battleground with the incoming shots
    def update_opponent_shots(self, grid, incoming_shots):
        if incoming_shots:
            for x, y in incoming_shots:
                grid[x][y] = 9
        return grid

    # function that counts the number of ships and her ports hit
    def ship_status(self, ships_dd):
        status = []
        for ship in ships_dd.keys():
            # as long as the default dictionary has a key, the ship is breathing
            if ship in [1, 2, 3, 4, 5]:
                status.append((ship, ships_dd[ship] / (ship ** 2)))
        return status

    # function to update the opponent battleground with the shots taken
    def update_enemy_dict(self, player_shots):
        if player_shots:
            for i, j in player_shots:
                self.enemy_dict[(i, j)] = -1


    # minesweeper matrix update
    def minesweeper(self, enemy_dict, player_shots):

        # up, down, right, left, upright, downright, up-left, down-left
        minesweeper = [(-1, 0), (1, 0), (0, 1), (0, -1), (-1, 1), (1, 1), (-1, -1), (1, -1)]

        # updating the enemy dictionary
        # only if it has hit!
        for i, j in player_shots:
            # opponent_battleground[i][j] = -1
            for x, y in minesweeper:
                # checking if the next position is in the board
                if (x + i) >= 0 and (x + i) <= 9 and (y + j) >= 0 and (y + j) <= 9:
                    # checking if it has already not been hit
                    if enemy_dict[(i + x, j + y)] != -1:
                        enemy_dict[(i + x, j + y)] += 1

        return enemy_dict

    # shot generator based on the minesweeper
    def shot_generator(self, enemy_dict, num=5):
        sorted_shot_list = sorted([(i[1], i[0]) for i in enemy_dict.items()], reverse=True)
        for i in range(num):
            yield sorted_shot_list[i][1]

    # function that randomly selects positions to hit and removes them
    def random_selection(self, enemy_dict, num=5):
        sorted_shot_list = sorted([(i[1], i[0]) for i in enemy_dict.items() if i[1] >= 0], reverse=True)
        shot = Player.random.sample(sorted_shot_list, k=num)
        shot_list = [i[1] for i in shot]

        return shot_list

In [3]:
p_goose = Player()

In [5]:
p_goose.setup_ships()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 5, 3, 0, 0, 0, 2],
 [0, 0, 0, 0, 5, 3, 1, 0, 0, 0],
 [4, 0, 0, 0, 5, 3, 0, 0, 0, 0],
 [4, 0, 0, 0, 5, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 5, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [8]:
p_goose.take_turn(history)

[(8, 2), (9, 4), (8, 4), (8, 1), (7, 2)]

In [19]:
#sorted([(i[1], i[0]) for i in p_goose.enemy_dict.items()], reverse = True)

In [7]:
history = [{'shots': [(9, 3), (8, 3), (7, 1), (9, 5), (9, 2)],
  'hits': 2,
  'incoming': [(2, 1), (0, 0), (6, 5), (1, 1), (3, 5)]}]

In [6]:
p_goose.take_turn([])

[(0, 3), (8, 5), (6, 7), (9, 9), (5, 7)]

In [9]:
p_goose.status

[(2, 1.0), (5, 1.0), (3, 0.6666666666666666), (1, 1.0), (4, 1.0)]

In [17]:
#p_goose.enemy_dict

In [36]:
opponent_battleground = [[0 for i in range(10)] for j in range(10)]

In [37]:
opponent_battleground

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [5]:
potential_list = []
for i in range(10):
    for j in range (10):
        potential_list.append((i,j))

In [7]:
potential_list = []

In [12]:
# Update potential list to this
potential_list = [(i,j) for i in range(10) for j in range(10)]

In [13]:
t_dict = {(i,j) for i in range(10) for j in range(10)}

In [91]:
test_dict = {(i,j):0 for i in range(10) for j in range(10)}

In [18]:
# initializing enemy dictionary
enemy_dict = {}
for i,j in potential_list:
    enemy_dict[(i,j)] = 0

In [19]:
history = [{'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(4, 5), (3, 4), (2, 3), (3, 2), (4, 2)]}]

In [24]:
for i,j in history[-1]['shots']:
    print((i,j))

(1, 3)
(6, 3)
(5, 4)
(9, 2)
(9, 8)


In [38]:
# for visualization
for i,j in history[-1]['shots']:
    opponent_battleground[i][j] = -1 

In [39]:
opponent_battleground

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
 [0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, -1, 0, 0, 0, 0, 0, -1, 0]]

In [28]:
# update the dictionary with the shots taken
for i,j in history[-1]['shots']:
    enemy_dict[(i,j)] = -1

In [30]:
if history[-1]['hits']:
    pass

In [25]:
# Using the minesweeper technique
# check if the next position is out of the board
# if the spot has been taken, update to -1
# check the number of hits
# if the surrounding location is not -1 then add one
# sort the dictionary out by keys in descending order
# return the top 5 keys to shoot

In [32]:
# up, down, right, left, upright, downright, upleft, downleft
minesweeper = [(-1,0), (1,0), (0,1), (0,-1), (-1,1), (1,1), (-1,-1), (1,-1)]

In [33]:
minesweeper

[(-1, 0), (1, 0), (0, 1), (0, -1), (-1, 1), (1, 1), (-1, -1), (1, -1)]

In [40]:
for i,j in history[-1]['shots']:
    #opponent_battleground[i][j] = -1 
    for x,y in minesweeper:
        # checking if the next position is in the board
        if (x+i) >= 0 and (x+i) <= 9 and (y+j) >=0 and (y+j) <=9:
            # checking if it has already not been hit
            if opponent_battleground[i+x][j+y] != -1:
                opponent_battleground[i+x][j+y] += 1

In [41]:
opponent_battleground

[[0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, -1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
 [0, 0, 1, 2, -1, 1, 0, 0, 0, 0],
 [0, 0, 1, -1, 2, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [0, 1, -1, 1, 0, 0, 0, 1, -1, 1]]

In [44]:
# updating the enemy dictionary
for i,j in history[-1]['shots']:
    #opponent_battleground[i][j] = -1 
    for x,y in minesweeper:
        # checking if the next position is in the board
        if (x+i) >= 0 and (x+i) <= 9 and (y+j) >=0 and (y+j) <=9:
            # checking if it has already not been hit
            if enemy_dict[(i+x, j+y)] != -1:
                enemy_dict[(i+x, j+y)] += 1

In [51]:
sorted_x = sorted([row for row in enemy_dict.values()], reverse = True)

In [76]:
def shot_generator(enemy_dict, num=5):
    sorted_shot_list = sorted([(i[1], i[0]) for i in enemy_dict.items()], reverse = True)
    for i in range(num):
        yield sorted_shot_list[i][1]

In [80]:
# this is a clever way to get the highest number of probable area first
sorted_y = sorted([(i[1], i[0]) for i in enemy_dict.items() if i[1] >= 0], reverse = True)

In [74]:
sorted_y[0][1]

(6, 4)

In [77]:
[number for number in shot_generator(enemy_dict, 5)]

[(6, 4), (5, 3), (9, 9), (9, 7), (9, 3)]

In [78]:
history = [{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]}]

In [82]:
# for visualization
for i,j in history[-1]['shots']:
    opponent_battleground[i][j] = -1 
opponent_battleground

[[0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, -1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, -1, 0, -1],
 [0, 0, 1, 2, -1, 1, 0, 0, 0, 0],
 [0, 0, 1, -1, 2, 1, 0, 0, 0, 0],
 [0, -1, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [0, 1, -1, 1, 0, 0, 0, 1, -1, 1]]

In [83]:
for i,j in history[-1]['shots']:
    #opponent_battleground[i][j] = -1 
    for x,y in minesweeper:
        # checking if the next position is in the board
        if (x+i) >= 0 and (x+i) <= 9 and (y+j) >=0 and (y+j) <=9:
            # checking if it has already not been hit
            if opponent_battleground[i+x][j+y] != -1:
                opponent_battleground[i+x][j+y] += 1

In [84]:
opponent_battleground

[[0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, -1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 2, -1, 3, 1],
 [0, 0, 0, 1, 1, 1, 2, -1, 3, -1],
 [0, 0, 1, 2, -1, 1, 1, 1, 2, 1],
 [1, 1, 2, -1, 2, 1, 0, 0, 0, 0],
 [1, -1, 2, 1, 1, 0, 0, 0, 0, 0],
 [1, 3, 3, 2, 0, 0, 0, 1, 1, 1],
 [0, 2, -1, 2, 0, 0, 0, 1, -1, 1]]

In [85]:
# update the dictionary with the shots taken
for i,j in history[-1]['shots']:
    enemy_dict[(i,j)] = -1

# updating the enemy dictionary 
# only if it has hit!
for i,j in history[-1]['shots']:
    #opponent_battleground[i][j] = -1 
    for x,y in minesweeper:
        # checking if the next position is in the board
        if (x+i) >= 0 and (x+i) <= 9 and (y+j) >=0 and (y+j) <=9:
            # checking if it has already not been hit
            if enemy_dict[(i+x, j+y)] != -1:
                enemy_dict[(i+x, j+y)] += 1   

In [86]:
# should be getting (8,1) (8,2) (3,8) (4,8)
[number for number in shot_generator(enemy_dict, 5)]

[(8, 2), (8, 1), (4, 8), (3, 8), (9, 3)]

In [10]:
sorted_xy = sorted([(i[1], i[0]) for i in p_goose.enemy_dict.items()], reverse = True)

In [11]:
sorted_xy[:7]

[(4, (8, 2)),
 (3, (9, 4)),
 (3, (8, 4)),
 (2, (8, 1)),
 (2, (7, 2)),
 (1, (9, 6)),
 (1, (9, 1))]

In [13]:
random.sample(sorted_xy[:7], k=10)

ValueError: Sample larger than population or is negative

In [28]:
m = [(1,2),(2,3),(4,5),(6,7)]

In [29]:
random.choice(m)

(6, 7)

In [30]:
random.sample(m, k=2)

[(1, 2), (2, 3)]

In [35]:
shot = [(0, (7, 2)), (0, (8, 3)), (0, (1, 0)), (0, (1, 6)), (0, (6, 0))]

In [36]:
shot_list = [i[1] for i in shot]

In [37]:
shot_list

[(7, 2), (8, 3), (1, 0), (1, 6), (6, 0)]